## CAPSTONE B Intrusion Detection flow based  Classification using RNN -BLSTM try 

In [1]:
%config IPCompleter.greedy=True
import pandas as pd
import seaborn as sns
import numpy as np
import tensorflow as tf 
import matplotlib as matplot
import matplotlib.pyplot as plt
%matplotlib inline

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import warnings
warnings.filterwarnings("ignore")

from keras.models import Model, load_model
from keras.layers import *
from keras.callbacks import ModelCheckpoint, TensorBoard
from keras import regularizers

from sklearn.metrics import (confusion_matrix, precision_recall_curve, auc,
                             roc_curve, recall_score, classification_report, f1_score,
                             precision_recall_fscore_support)
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, VotingClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder,normalize

Using TensorFlow backend.


Preprocessing -Transformation and Scaling 

In [2]:
#tf version
import tensorflow as tf
tf.__version__
#tensorboard version
from tensorboard import version; print(version.VERSION)
#Keras version
import keras 
keras.__version__

'1.14.0'

1.14.0


'2.3.1'

In [3]:
import os
import pandas as pd
os.chdir('C:\\Users\\Brook\\Desktop\\#SMU_Courses\\#MSDS 6130 Capstone A & B\\MSDS6120 CapstoneA\\#ProjectData&NoteBook')

In [4]:
print ( "Current working dir : %s" % os.getcwd())

Current working dir : C:\Users\Brook\Desktop\#SMU_Courses\#MSDS 6130 Capstone A & B\MSDS6120 CapstoneA\#ProjectData&NoteBook


In [5]:
pd.set_option('display.max_columns', None)

In [6]:
train = pd.read_csv(r'UNSW_NB15_training_set.csv')
test = pd.read_csv(r'UNSW_NB15_testing_set.csv')
list_events = pd.read_csv(r'UNSW_NB15_LIST_EVENTS.csv')
features = pd.read_csv(r'UNSW_NB15_features.csv',encoding='latin1')

In [7]:
combined_data = pd.concat([train, test]).drop(['id'],axis=1)

Function to encode string features

In [8]:
le1 = LabelEncoder()
le = LabelEncoder()

vector = combined_data['attack_cat']

print("attack cat:", set(list(vector))) # use print to make it print on single line 

combined_data['attack_cat'] = le1.fit_transform(vector)
combined_data['proto'] = le.fit_transform(combined_data['proto'])
combined_data['service'] = le.fit_transform(combined_data['service'])
combined_data['state'] = le.fit_transform(combined_data['state'])

vector = combined_data['attack_cat']
print('\nDescribing attack_type: ')
print("min", vector.min())
print("max", vector.max())
print("mode",vector.mode())
print("mode", len(np.where(vector.values==6)[0])/len(vector),"%")
print("looks like 6 is 'normal', but its not that common")

attack cat: {'Shellcode', 'Worms', 'Backdoor', 'DoS', 'Normal', 'Analysis', 'Generic', 'Reconnaissance', 'Fuzzers', 'Exploits'}

Describing attack_type: 
min 0
max 9
mode 0    6
dtype: int32
mode 0.3609225646458884 %
looks like 6 is 'normal', but its not that common


In [9]:
le1.inverse_transform([0,1,2,3,4,5,6,7,8,9,])

array(['Analysis', 'Backdoor', 'DoS', 'Exploits', 'Fuzzers', 'Generic',
       'Normal', 'Reconnaissance', 'Shellcode', 'Worms'], dtype=object)

???? Dropping thr features of Low Standard deviation and Low corr

In [10]:
##?????
lowSTD = list(combined_data.std().to_frame().nsmallest(6,columns=0).index)
lowCORR = list(combined_data.corr().abs().sort_values('attack_cat')['attack_cat'].nsmallest(3).index) # .where(lambda x: x < 0.005).dropna()
drop = set( lowCORR + lowSTD)
#drop = {'ackdat', 'ct_ftp_cmd', 'djit', 'is_ftp_login', 'is_sm_ips_ports', 'response_body_len', 'sjit', 'synack', 'tcprtt'}
combined_data.shape
combined_data_reduced=combined_data.drop(drop,axis=1)
combined_data_reduced.shape

(257673, 44)

(257673, 35)

In [11]:
pd.set_option('display.max_columns', None)
combined_data_reduced.head()

,dur,proto,service,state,spkts,dpkts,sbytes,dbytes,rate,sttl,dttl,sload,dload,sloss,dloss,sinpkt,dinpkt,swin,stcpb,dtcpb,dwin,smean,dmean,trans_depth,ct_srv_src,ct_state_ttl,ct_dst_ltm,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,ct_flw_http_mthd,ct_src_ltm,ct_srv_dst,attack_cat,label
0,0.121478,113,0,4,6,4,258,172,74.087490,252,254,14158.942380,8495.365234,0,0,24.295600,8.375000,255,621772692,2202533631,255,43,43,0,1,0,1,1,1,1,0,1,1,6,0
1,0.649902,113,0,4,14,38,734,42014,78.473372,62,252,8395.112305,503571.312500,2,17,49.915000,15.432865,255,1417884146,3077387971,255,52,1106,0,43,1,1,1,1,2,0,1,6,6,0
2,1.623129,113,0,4,8,16,364,13186,14.170161,62,252,1572.271851,60929.230470,1,6,231.875571,102.737203,255,2116150707,2963114973,255,46,824,0,7,1,2,1,1,3,0,2,6,6,0
3,1.681642,113,3,4,12,12,628,770,13.677108,62,252,2740.178955,3358.622070,1,3,152.876547,90.235726,255,1107119177,1047442890,255,52,64,0,1,1,2,1,1,3,0,2,1,6,0
4,0.449454,113,0,4,10,6,534,268,33.373826,254,252,8561.499023,3987.059814,2,1,47.750333,75.659602,255,2436137549,1977154190,255,53,45,0,43,1,2,2,1,40,0,2,39,6,0


In [12]:
lowSTD 

['ackdat', 'synack', 'tcprtt', 'is_ftp_login', 'ct_ftp_cmd', 'is_sm_ips_ports']

In [13]:
#https://stats.stackexchange.com/questions/309612/removing-features-with-low-variance-in-classification-models
lowCORR

['sjit', 'response_body_len', 'djit']

In [14]:
## ??????
transform = list(combined_data_reduced.columns.values[4:])
transform

['spkts',
 'dpkts',
 'sbytes',
 'dbytes',
 'rate',
 'sttl',
 'dttl',
 'sload',
 'dload',
 'sloss',
 'dloss',
 'sinpkt',
 'dinpkt',
 'swin',
 'stcpb',
 'dtcpb',
 'dwin',
 'smean',
 'dmean',
 'trans_depth',
 'ct_srv_src',
 'ct_state_ttl',
 'ct_dst_ltm',
 'ct_src_dport_ltm',
 'ct_dst_sport_ltm',
 'ct_dst_src_ltm',
 'ct_flw_http_mthd',
 'ct_src_ltm',
 'ct_srv_dst',
 'attack_cat',
 'label']

In [15]:
transform.append('dur')
transform.remove('attack_cat')
# transform min-max norm 
combined_data_reduced[transform] = combined_data_reduced[transform].apply(lambda x: (x - x.min()) / (x.max() - x.min()))

Combined data reducd transformation

In [16]:
combined_data_reduced[transform]

,spkts,dpkts,sbytes,dbytes,rate,sttl,dttl,sload,dload,sloss,dloss,sinpkt,dinpkt,swin,stcpb,dtcpb,dwin,smean,dmean,trans_depth,ct_srv_src,ct_state_ttl,ct_dst_ltm,ct_src_dport_ltm,ct_dst_sport_ltm,ct_dst_src_ltm,ct_flw_http_mthd,ct_src_ltm,ct_srv_dst,label,dur
0,0.000470,0.000363,0.000016,0.000012,0.000074,0.988235,1.000000,2.364553e-06,0.000379,0.000000,0.000000,2.879598e-04,0.000145,1.0,0.144768,0.512828,1.0,0.012838,0.028667,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,2.024634e-03
1,0.001221,0.003449,0.000049,0.002866,0.000078,0.243137,0.992126,1.401989e-06,0.022458,0.000376,0.003087,5.916098e-04,0.000267,1.0,0.330128,0.716524,1.0,0.018919,0.737333,0.0,0.677419,0.166667,0.000000,0.000000,0.0,0.015625,0.0,0.000000,0.081967,0.0,1.083170e-02
2,0.000658,0.001452,0.000024,0.000900,0.000014,0.243137,0.992126,2.625704e-07,0.002717,0.000188,0.001090,2.748269e-03,0.001779,1.0,0.492706,0.689918,1.0,0.014865,0.549333,0.0,0.096774,0.166667,0.017241,0.000000,0.0,0.031250,0.0,0.016949,0.081967,0.0,2.705215e-02
3,0.001033,0.001089,0.000042,0.000053,0.000014,0.243137,0.992126,4.576117e-07,0.000150,0.000188,0.000545,1.811945e-03,0.001563,1.0,0.257772,0.243882,1.0,0.018919,0.042667,0.0,0.000000,0.166667,0.017241,0.000000,0.0,0.031250,0.0,0.016949,0.000000,0.0,2.802737e-02
4,0.000845,0.000545,0.000036,0.000018,0.000033,0.996078,0.992126,1.429776e-06,0.000178,0.000376,0.000182,5.659534e-04,0.001310,1.0,0.567209,0.460351,1.0,0.019595,0.030000,0.0,0.677419,0.166667,0.017241,0.017241,0.0,0.609375,0.0,0.016949,0.622951,0.0,7.490901e-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82327,0.000094,0.000000,0.000006,0.000000,0.200000,0.996078,0.000000,1.389445e-02,0.000000,0.000000,0.000000,5.926172e-08,0.000000,0.0,0.000000,0.000000,0.0,0.018919,0.000000,0.0,0.000000,0.333333,0.017241,0.000000,0.0,0.015625,0.0,0.016949,0.000000,0.0,8.333335e-08
82328,0.001785,0.000726,0.001257,0.000024,0.000024,0.996078,0.992126,2.072552e-05,0.000100,0.001316,0.000182,6.623096e-04,0.002489,1.0,0.249720,0.764699,1.0,0.593919,0.029333,0.0,0.000000,0.166667,0.017241,0.000000,0.0,0.000000,0.0,0.033898,0.016393,0.0,1.843502e-02
82329,0.000000,0.000000,0.000002,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,7.111492e-01,0.000000,0.0,0.000000,0.000000,0.0,0.014865,0.000000,0.0,0.000000,0.333333,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000e+00
82330,0.000000,0.000000,0.000002,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,7.111493e-01,0.000000,0.0,0.000000,0.000000,0.0,0.014865,0.000000,0.0,0.000000,0.333333,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000e+00


In [17]:
data_x = combined_data_reduced.drop(['attack_cat','label'], axis=1) # droped label
data_y = combined_data_reduced.loc[:,['label']]

# del combined_data # free mem
# Use stratification to fix imbalance of normal vs. attack categories

# we changed to 60% , 40% as the epoch is very slow for BLSTM 


X_train, X_test, y_train, y_test = train_test_split(data_x, data_y, train_size=0.6,test_size=0.4, random_state = 1, stratify = data_y)

#https://www.bitdegree.org/learn/train-test-split
#X_train, X_test, y_train, y_test = train_test_split(data_x, data_y, test_size=.70, random_state=42) # TODO

In [18]:
#combined_data_reduced[transform].head(5)

In [19]:
#data_x

In [20]:
#data_y

In [21]:
data_y.isnull().values.any()

False

In [22]:
is_all_zero = np.all((data_y == 0))
if is_all_zero:
    print('Array contains only 0')
else:
    print('Array has non-zero items too')


Array has non-zero items too


In [23]:
# after we split to 70 /30 to combine dataset

X_train.shape
y_train.shape
X_test.shape # test is larger... good 
y_test.shape

(154603, 33)

(154603, 1)

(103070, 33)

(103070, 1)

Benchmark before additional feature removal

In [24]:
#import xgboost 
#import lightgbm

# DTC = DecisionTreeClassifier()
# RFC = RandomForestClassifier(n_estimators=150, random_state=42, n_jobs=-1)
# ETC = ExtraTreesClassifier(n_estimators=200, random_state=42, n_jobs=-1)

# Feature selection methods deployed , DTC , RFC , ETC 
DTC = DecisionTreeClassifier() 
RFC = RandomForestClassifier(n_estimators=25, random_state=1)
ETC = ExtraTreesClassifier(n_estimators=10, criterion='gini', max_features='auto', bootstrap=False)


#XGB = xgboost.XGBClassifier(n_estimators=150, n_jobs=-1)
#GBM = lightgbm.LGBMClassifier(objective='binary', n_estimators= 500) # multiclass

list_of_CLFs_names = []
list_of_CLFs = [DTC, RFC, ETC]
#list_of_CLFs = [DTC, RFC, ETC, XGB, GBM]
ranking = []

for clf in list_of_CLFs:
    _ = clf.fit(X_train,y_train)
    pred = clf.score(X_test,y_test)
    name = str(type(clf)).split(".")[-1][:-2]
    print("Acc: %0.5f for the %s" % (pred, name))

    ranking.append(pred)
    list_of_CLFs_names.append(name)

Acc: 0.93582 for the DecisionTreeClassifier
Acc: 0.94784 for the RandomForestClassifier
Acc: 0.94372 for the ExtraTreesClassifier


In [25]:
# mlxtend error fix,https://stackoverflow.com/questions/49889524/issues-importing-mlxtend-python
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, VotingClassifier
from mlxtend.classifier import EnsembleVoteClassifier

https://medium.com/@jonathan_hui/machine-learning-singular-value-decomposition-svd-principal-component-analysis-pca-1d45e885e491

https://machinelearningmastery.com/singular-value-decomposition-for-machine-learning/

In [26]:
eclf = EnsembleVoteClassifier

In [27]:

#refe
#https://github.com/alik604/cyber-security/blob/master/Intrusion-Detection/UNSW_NB15.ipynb

#http://rasbt.github.io/mlxtend/user_guide/classifier/EnsembleVoteClassifier/


#eclf = VotingClassifier(clfs=list_of_CLFs, voting='soft')  #refit=False
eclf = EnsembleVoteClassifier(clfs=list_of_CLFs, voting='soft')  #refit=False
_ = eclf.fit(X_train, y_train)
pred = eclf.score(X_test, y_test)
print("Acc: %0.5f for the %s" % (pred, str(type(eclf)).split(".")[-1][:-2]))
pred = eclf.predict(X_test)
probas = eclf.predict_proba(X_test)
plt.show()



Acc: 0.94363 for the EnsembleVoteClassifier


In [28]:
from sklearn.feature_selection import RFE
from sklearn.decomposition import PCA,TruncatedSVD,PCA
from sklearn.svm import LinearSVC
# Number of trees
n = 10

In [29]:
#Try RFE 

rfe = RFE(DecisionTreeClassifier(), n).fit(X_train, y_train)

desiredIndices = np.where(rfe.support_==True)[0]
whitelist = X_train.columns.values[desiredIndices]
X_train_RFE, X_test_RFE = X_train[whitelist], X_test[whitelist]

print('new shape', X_train_RFE.shape) 

for clf in list_of_CLFs:
    _ = clf.fit(X_train_RFE,y_train)
    pred = clf.score(X_test_RFE,y_test)
    name = str(type(clf)).split(".")[-1][:-2]
    print("Acc: %0.5f for the %s" % (pred, name))

    ranking.append(pred)
    list_of_CLFs_names.append(name)


eclf = EnsembleVoteClassifier(clfs=list_of_CLFs, voting='soft')  # refit=False,
_ = eclf.fit(X_train_RFE, y_train)
pred = eclf.score(X_test_RFE, y_test)
print("Acc: %0.5f for the %s" % (pred, str(type(eclf)).split(".")[-1][:-2]))


pred = eclf.predict(X_test_RFE)
probas = eclf.predict_proba(X_test_RFE)
plt.show()


new shape (154603, 10)
Acc: 0.93326 for the DecisionTreeClassifier
Acc: 0.94543 for the RandomForestClassifier
Acc: 0.94063 for the ExtraTreesClassifier
Acc: 0.94023 for the EnsembleVoteClassifier


In [30]:
X_train_RFE.head(5)

,dur,sbytes,dbytes,sttl,sloss,dinpkt,stcpb,smean,ct_dst_src_ltm,ct_srv_dst
63012,1.666667e-07,0.000006,0.000000,0.996078,0.000000,0.000000e+00,0.000000,0.022297,0.031250,0.163934
83653,1.556752e-02,0.000049,0.000207,0.243137,0.000376,1.983197e-03,0.268483,0.032432,0.000000,0.000000
146586,5.000001e-08,0.000006,0.000000,0.996078,0.000000,0.000000e+00,0.000000,0.022297,0.343750,0.360656
149380,1.500000e-07,0.000006,0.000000,0.996078,0.000000,0.000000e+00,0.000000,0.022297,0.640625,0.672131
67201,6.682085e-03,0.000206,0.000053,0.996078,0.000000,5.195773e-08,0.000000,0.992568,0.015625,0.016393


In [31]:
# Try SVD 

svd = TruncatedSVD(n_components=n).fit(X_train)
X_train_svd, X_test_svd = svd.transform(X_train), svd.transform(X_test)

for clf in list_of_CLFs:
    _ = clf.fit(X_train_svd, y_train)
    pred = clf.score(X_test_svd, y_test)
    name = str(type(clf)).split(".")[-1][:-2]
    print("Acc: %0.5f for the %s" % (pred, name))

    ranking.append(pred)
    list_of_CLFs_names.append(name)

eclf = EnsembleVoteClassifier(clfs=list_of_CLFs,  voting='soft') #refit=False,
_ = eclf.fit(X_train_svd, y_train)
pred = eclf.score(X_test_svd, y_test)
print("Acc: %0.5f for the %s" % (pred, str(type(eclf)).split(".")[-1][:-2]))


pred = eclf.predict(X_test_svd)
probas = eclf.predict_proba(X_test_svd)
plt.show()

Acc: 0.90997 for the DecisionTreeClassifier
Acc: 0.92177 for the RandomForestClassifier
Acc: 0.91584 for the ExtraTreesClassifier
Acc: 0.91608 for the EnsembleVoteClassifier


In [32]:
svd = TruncatedSVD(n_components=n-1).fit(X_train_RFE)   
X_train_svd, X_test_svd = svd.transform(X_train_RFE), svd.transform(X_test_RFE)

Benchmark after additional feature removal

In [33]:

DTC = DecisionTreeClassifier() 
RFC = RandomForestClassifier(n_estimators=25, random_state=1)
ETC = ExtraTreesClassifier(n_estimators=10, criterion='gini', max_features='auto', bootstrap=False)
eclf = VotingClassifier(estimators=[('DecisionTreeClassifier', DTC), ('RandomForestClassifier', RFC),('ExtraTreesClassifier',ETC)], voting='hard')
for clf, label in zip([DTC, RFC,ETC, eclf], ['DecisionTreeClassifier', 'RandomForestClassifier', 'ExtraTreesClassifier', 'Ensemble']): 
        _ = eclf.fit(X_train_svd,y_train)
        pred = eclf.score(X_test_svd,y_test)
        print("Acc: %0.10f [%s]" % (pred,label))

Acc: 0.9297176676 [DecisionTreeClassifier]
Acc: 0.9302512855 [RandomForestClassifier]
Acc: 0.9299893276 [ExtraTreesClassifier]
Acc: 0.9303386048 [Ensemble]


In [34]:
print(X_train_svd.shape)

(154603, 9)


In [35]:
#X_train_svd

#### Deep Learning 

We ran  SVD/PCA to remove variance since the  goal to "get the better deep learning result possible"

In [36]:
from keras.models import Sequential


In [37]:
# ?? why transformation here

from sklearn.preprocessing import MinMaxScaler
mms = MinMaxScaler()
X_train_svd = mms.fit_transform(X_train_svd)
X_test_svd = mms.transform(X_test_svd)

In [38]:
X = X_train_svd
X_t = X_test_svd

dim = X.shape[1]
dim

9

In [39]:
X.shape   
y_train.shape
X_t.shape 
y_test.shape


(154603, 9)

(154603, 1)

(103070, 9)

(103070, 1)

## BLSTM modification zone 

## Changing the dimension from 2 to 3 to run BLSTM 

In [40]:
# varriable mapping for RNN and BLSTM 

# X_v is X_train  
# y_v is y_train 
# X  is X_train_svd
# X_t  is X_test_svd
# y_test is y_test

In [41]:
#training data set mash up to 3D

In [42]:

#X_v ,y_v = X_t[:X_t.shape[0]/5], y_test[:X_t.shape[0]/5]

X_v,y_v  = X[:X.shape[0]], y_train[:y_train.shape[0]]


In [43]:
#y_v

In [44]:
#X_v

In [45]:
# Changing the train data set to 3 dimensional for BLSTM

X_v =[]
y_v =[]

# Why do we select 60 time stamps???

for i in range(60,X.shape[0] ):  
    X_v.append(X[i-60:i])
    y_v.append(X[i,0])
       

In [46]:
#X_train, y_train = np.array(X_train), np.array(y_train)

#X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1],1))

# X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

X_v, y_v = np.array(X_v), np.array(y_v)


In [47]:
X_v.shape, y_v.shape


((154543, 60, 9), (154543,))

### Validation data set mash up is done after epoch run as it causes slow epoch run 

## Define The Neural Network Model

In [77]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import TimeDistributed
from keras.layers import Bidirectional
from keras.layers import LSTM
from keras import regularizers   # to improve loss not come nan

In [83]:
# BLSTM model 

regressior = Sequential()

regressior.add(Bidirectional(LSTM(units =60,activation = 'relu', return_sequences = True),input_shape = (X_v.shape [1],9)))  
regressior.add(Dropout(0.07))


regressior.add(Bidirectional(LSTM(units =60,activation = 'relu', kernel_regularizer=regularizers.l2(0.0001),return_sequences = True) ))
regressior.add(Dropout(0.07))

regressior.add(Bidirectional(LSTM(units =80,activation = 'relu', kernel_regularizer=regularizers.l2(0.0001) ,return_sequences = True) ))
regressior.add(Dropout(0.07))

regressior.add(Bidirectional(LSTM(units =90,activation = 'sigmoid',kernel_regularizer=regularizers.l2(0.0001))))
regressior.add(Dropout(0.07))

regressior.add(Dense(units = 1))  # change 10 to 1 for binary LSTM and run the neural net model from the library 
 

In [84]:
# #  RNN Model 

# classifier = Sequential()
# #First Hidden Layer
# classifier.add(Dense(18, activation='relu', input_dim=dim))
# classifier.add(Dense(36, activation='relu'))
# classifier.add(Dropout(0.07))
# classifier.add(Dense(18, activation='relu' ))
# classifier.add(Dropout(0.07))
# classifier.add(Dense(9, activation='relu'))

# classifier.add(Dense(1, activation='sigmoid'))

# classifier.compile(optimizer ='adam',loss='binary_crossentropy', metrics =['accuracy'])

In [85]:

# RNN tensor board  

# tensorboard = TensorBoard(log_dir=f".\logs\MODEL", histogram_freq=1,write_graph=True)

In [86]:
# BLSTM regressor summary 

regressior.summary()


Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bidirectional_21 (Bidirectio (None, 60, 120)           33600     
_________________________________________________________________
dropout_21 (Dropout)         (None, 60, 120)           0         
_________________________________________________________________
bidirectional_22 (Bidirectio (None, 60, 120)           86880     
_________________________________________________________________
dropout_22 (Dropout)         (None, 60, 120)           0         
_________________________________________________________________
bidirectional_23 (Bidirectio (None, 60, 160)           128640    
_________________________________________________________________
dropout_23 (Dropout)         (None, 60, 160)           0         
_________________________________________________________________
bidirectional_24 (Bidirectio (None, 180)             

In [87]:
# BLSTM Model note

# https://machinelearningmastery.com/adam-optimization-algorithm-for-deep-learning/


# Adam Configuration Parameters

# alpha. Also referred to as the learning rate or step size. The proportion that weights are updated (e.g. 0.001). 
# Larger values (e.g. 0.3) results in faster initial learning before the rate is updated. Smaller values (e.g. 1.0E-5)
# slow learning right down during training
# beta1. The exponential decay rate for the first moment estimates (e.g. 0.9).
# beta2. The exponential decay rate for the second-moment estimates (e.g. 0.999). This value should be set close to
# 1.0 on problems with a sparse gradient (e.g. NLP and computer vision problems).
# epsilon. Is a very small number to prevent any division by zero in the implementation (e.g. 10E-8).

# The Adam paper suggests:

# Good default settings for the tested machine learning problems are alpha=0.001, beta1=0.9, beta2=0.999 and 
# epsilon=10−8

# The TensorFlow documentation suggests some tuning of epsilon:

# The default value of 1e-8 for epsilon might not be a good default in general. For example, when training an 
# Inception network on ImageNet a current good choice is 1.0 or 0.1.

In [89]:

# BLSTM model 

from tensorflow import keras
from tensorflow.keras import optimizers

adam= optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)

#regressior.compile(optimizer = 'adam',loss = 'binary_crossentropy' ) 

# regressior.compile(optimizer = 'adam',loss = 'sparse_categorical_crossentropy', metrics=['accuracy']))

regressior.compile(optimizer = 'adam',loss = 'mean_squared_error')

#opt = keras.optimizers.Adam(learning_rate=0.01)

In [90]:
#Learning rate 

# The amount that the weights are updated during training is referred to as the step size or the “learning rate.” 
# Specifically, the learning rate is a configurable hyperparameter used in the training of neural networks that has 
# a small positive value, often in the range between 0.0 and 1.0

In [91]:
# BLSTM Model 

#train test validation is too much hour 

#history = regressior.fit(X_v, y_v, batch_size = 10, epochs = 1, validation_data=(X_t1, y_test) ).history

history = regressior.fit(X_v, y_v, batch_size = 10, epochs = 1 ).history

Epoch 1/1
154543/154543 [==============================] - 2258s 15ms/step - loss: 0.0873


## Validation data set mash up to 3D

In [92]:
# varriable assignment 

X_t1,y_test  = X_t[:X_t.shape[0]], y_test[:y_test.shape[0]]

# Changing the validation data set to 3 dimensional for BLSTM

X_t1 =[]
y_test=[]

# Why do we select 60 time stamps???

for i in range(60,X_t.shape[0] ):  
    X_t1.append(X_t[i-60:i])
    y_test.append(X_t[i,0])
    
    
# change to numpy array 

#X_train, y_train = np.array(X_train), np.array(y_train)

#X_train = np.reshape(X_train, (X_train.shape[0], X_train

X_t1, y_test = np.array(X_t1), np.array(y_test)

# check the converted shape 

X_t1.shape, y_test.shape


# reshape array from 3d to 2d if needed 

((103010, 60, 9), (103010,))

In [93]:
# RNN Model classification out put 


# Data voloume 77301 dataset
# the name tf.summary is deprecated. please use tf.compat.v1.summary instead
# if you get invalid argument error when run the model use 
#tf.reset_default_graph() 
#https://stackoverflow.com/questions/51872592/invalidargumenterror-you-must-feed-a-value-for-placeholder-tensor-placeholder

# from tensorflow.keras import layers, Sequential

# history = classifier.fit(X,y_train, batch_size=16, epochs=10, validation_data=(X_t,y_test) , callbacks=[tensorboard]).history

In [94]:
# varriable mapping for RNN and BLSTM 

# X_v is X_train  
# y_v is y_train 
# X  is X_train_svd
# X_t  is X_test_svd
# y_test is y_test

In [95]:
# RNN

#pip install keras_sequential_ascii
# from keras_sequential_ascii import keras2ascii
# keras2ascii(classifier)

In [96]:
# RNN 

#eval_model=classifier.evaluate(X, y_train )
#print(eval_model)


In [97]:
#BLSTM   y_train is the same like y_v but we take 60 rows out during dimension change 

# reshape x_v to 2d to make prediction 
#X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1],1))

eval_model=regressior.evaluate(X_v, y_v )
print(eval_model)

154543/154543 [==============================] - 348s 2ms/step
0.08373949081035402


In [98]:
# RNN 

#https://medium.com/datadriveninvestor/building-neural-network-using-keras-for-classification-3a3656c726c1
# eval_model=classifier.evaluate(X_t, y_test)
# print(eval_model)


In [99]:
#BLSTM 
eval_model=regressior.evaluate(X_t1, y_test)
print(eval_model)

103010/103010 [==============================] - 230s 2ms/step
0.08361336451377142


In [179]:
# RNN

#We now predict the output for our test dataset. If the prediction is greater than 0.5 then the output is 1 else the output is 0
#y_pred=classifier.predict(X_t)
#y_pred =(y_pred>0.5)

In [180]:
# BLSTM 

y_pred=regressior.predict(X_t1)
y_pred =(y_pred>0.5)


In [156]:
y_pred.shape


(103010, 1)

In [162]:
y_pred

array([[ True],
       [ True],
       [ True],
       ...,
       [ True],
       [ True],
       [ True]])

In [171]:
y_test

array([0.20426259, 0.74665767, 0.77469042, ..., 0.14910879, 0.25456511,
       0.70684149])

In [157]:
y_test.shape

(103010,)

In [173]:
# change y test to binary ( boolean )to prepare it for confusion matrix

def float_to_bool(y_test):
    if y_test > 0.5:
        return 'True'
    else :  
        'False'
        

In [159]:
y_test2 = np.ndarray(float_to_bool)(y_test).astype(bool)

TypeError: expected sequence object with len >= 0 or a single integer

In [160]:
y_test2

array([False,  True,  True, ..., False, False,  True])

In [161]:
y_test2.shape

(103010,)

In [ ]:
## ([False,  True,  True, True])  to      ([[ True],   [ True],   [ True]])  in python

In [152]:
y_test2

array([False,  True,  True, ..., False, False,  True])

In [183]:
y_test4 = np.array(y_test2)

In [184]:
y_test4

array([False,  True,  True, ..., False, False,  True])

In [167]:
#Convert a flat list to list of lists in python
# Calculate desired row/col
row = 103010
col = 1
y_test3= [y_test2[col*i : col*(i+1)] for i in range(row)]


In [168]:
import numpy as np
np.shape(y_test3)


(103010, 1)

In [182]:
#y_test3

[array([False]),
 array([ True]),
 array([ True]),
 array([ True]),
 array([ True]),
 array([False]),
 array([ True]),
 array([ True]),
 array([False]),
 array([ True]),
 array([ True]),
 array([ True]),
 array([False]),
 array([False]),
 array([ True]),
 array([False]),
 array([False]),
 array([ True]),
 array([ True]),
 array([ True]),
 array([ True]),
 array([ True]),
 array([ True]),
 array([False]),
 array([False]),
 array([False]),
 array([ True]),
 array([ True]),
 array([ True]),
 array([False]),
 array([ True]),
 array([False]),
 array([ True]),
 array([ True]),
 array([False]),
 array([ True]),
 array([False]),
 array([ True]),
 array([False]),
 array([ True]),
 array([False]),
 array([ True]),
 array([ True]),
 array([ True]),
 array([ True]),
 array([ True]),
 array([ True]),
 array([ True]),
 array([ True]),
 array([ True]),
 array([ True]),
 array([ True]),
 array([ True]),
 array([ True]),
 array([ True]),
 array([ True]),
 array([ True]),
 array([ True]),
 array([ True]

In [ ]:
#[array([False]), array([ True]) ]  vs  array([[ True] ,[ True]])

#[array([0]), array([ 1]) ]  vs  array([[ 1] ,[ 1]])

https://medium.com/datadriveninvestor/building-neural-network-using-keras-for-classification-3a3656c726c1

In [170]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test3, y_pred)
print(cm)

[[    0 35506]
 [    0 67504]]


In [ ]:
# plot confusion matrix 


In [ ]:
import numpy as np

def plot_confusion_matrix(cm,
                          target_names,
                          title='Confusion matrix',
                          cmap=None,
                          normalize=True):
    import matplotlib.pyplot as plt
    import numpy as np
    import itertools

    accuracy = np.trace(cm) / float(np.sum(cm))
    misclass = 1 - accuracy

    if cmap is None:
        cmap = plt.get_cmap('Blues')

    plt.figure(figsize=(8, 8))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title,fontsize=20 ,y=1.1)
    plt.colorbar()

    if target_names is not None:
        tick_marks = np.arange(len(target_names))
        plt.xticks(tick_marks, target_names, rotation=45)
        plt.yticks(tick_marks, target_names)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]


    thresh = cm.max() / 1.5 if normalize else cm.max() / 2
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        if normalize:
            plt.text(j, i, "{:0.4f}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")
        else:
            plt.text(j, i, "{:,}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")


    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label\naccuracy={:0.4f}; misclass={:0.4f}'.format(accuracy, misclass))
    plt.show()

In [ ]:
plot_confusion_matrix((cm),
                      normalize    = False,
                      target_names = ['Attack', 'Not Attack'],
                      title        = "Confusion Matrix")

In [ ]:
#https://www.geeksforgeeks.org/confusion-matrix-machine-learning/
# Constructing the confusion matrix.

#from sklearn.metrics import confusion_matrix
#confusion_matrix(y_test, y_pred)

In [ ]:
# Finding precision and recall
from sklearn.metrics import precision_score, recall_score
precision_score(y_test, y_pred)
recall_score(y_test, y_pred)

In [ ]:
# To compute the F1 score, simply call the f1_score() function:
from sklearn.metrics import f1_score
f1_score(y_test, y_pred)

In [ ]:
def accuracy(confusion_matrix):
    diagonal_sum = confusion_matrix.trace()
    sum_of_all_elements = confusion_matrix.sum()
    return diagonal_sum / sum_of_all_elements 


In [ ]:
accuracy(cm)

In [ ]:
mse = np.mean(np.power(X_test_svd - y_pred, 2), axis=1)
error_df = pd.DataFrame({'reconstruction_error': mse,'true_class': y_test.values.reshape(1,-1)[0]})
# error_df.describe()
print()
print(np.mean(mse))
#cm = confusion_matrix(y_test, y_pred)

#print(cm)

In [ ]:
plt.plot(history['loss'])
plt.plot(history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper right');

In [ ]:
fpr, tpr, thresholds = roc_curve(error_df.true_class, error_df.reconstruction_error,pos_label=1)
roc_auc = auc(fpr, tpr)


plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, label='AUC = %0.4f'% roc_auc)
plt.legend(loc='lower right')
plt.plot([0,1],[0,1],'r--')
plt.xlim([-0.001, 1])
plt.ylim([0, 1.001])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show();

In [ ]:
precision, recall, th = precision_recall_curve(error_df.true_class, error_df.reconstruction_error,pos_label=1)
plt.plot(recall, precision, 'b', label='Precision-Recall curve')
plt.title('Recall vs Precision')
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.show()

https://www.hindawi.com/journals/complexity/2019/6516253/